# Install Necessary Dependencies

In [4]:
!pip install tensorflow gym keras-rl2 gym[atari]

In [5]:
import gym 
import random

In [6]:
pip install gym[accept-rom-license]

Note: you may need to restart the kernel to use updated packages.


In [7]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

/home/makena/anaconda3/lib/python3.9/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SpaceInvaders-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]
/home/makena/anaconda3/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/makena/anaconda3/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [8]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [10]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render(mode='rgb_array')
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:105.0
Episode:2 Score:80.0
Episode:3 Score:135.0
Episode:4 Score:155.0
Episode:5 Score:135.0


In [9]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

2022-08-08 11:02:57.241357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-08 11:02:57.241462: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model


In [16]:
del model

In [17]:
model = build_model(height, width, channels, actions)

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten_1 (Flatten)         (None, 67584)             0         
                                                                 
 dense_4 (Dense)             (None, 512)               34603520  
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dense_6 (Dense)             (None, 6)                

In [19]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [20]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [21]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

/home/makena/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-08-08 11:05:47.042781: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-08-08 11:05:47.676888: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 138412032 exceeds 10% of free system memory.
2022-08-08 11:05:47.677947: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 138412032 exceeds 10% of free system memory.


In [ ]:
dqn.fit(env, nb_steps=500, visualize=False, verbose=2)

In [24]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

NameError: name 'dqn' is not defined

In [25]:
dqn.save_weights('SavedWeights/dqn_weights.h5f')

NameError: name 'dqn' is not defined

In [38]:
del model, dqn

In [22]:
dqn.load_weights('/home/makena/SavedWeights/dqn_weights.h5f')

In [23]:
scores = dqn.test(env, nb_episodes=5, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 5 episodes ...


/home/makena/anaconda3/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Episode 1: reward: 160.000, steps: 599
Episode 2: reward: 110.000, steps: 498
Episode 3: reward: 240.000, steps: 730
Episode 4: reward: 85.000, steps: 476
Episode 5: reward: 80.000, steps: 488
135.0
